Простой пример: Типы Деревьев в Лесу
Представим симуляцию леса, где нужно отобразить тысячи деревьев. У деревьев есть общие свойства (тип, цвет, текстура) и уникальные (координаты). Легковес поможет не хранить общие свойства для каждого дерева.

In [1]:
import json
from typing import Dict, Tuple

# 1. Flyweight Interface/Class (Легковес - Тип Дерева)
#    (Может быть и интерфейс, но здесь сразу конкретный класс)
class TreeType:
    """
    Легковес: хранит общее (intrinsic) состояние для типа дерева.
    Это состояние не должно меняться после создания.
    """
    def __init__(self, name: str, color: str, texture: str):
        print(f"FLYWEIGHT: Creating NEW TreeType - Name: {name}, Color: {color}, Texture: {texture}")
        self._name = name
        self._color = color
        self._texture = texture # Представим, что это "тяжелые" данные текстуры

    def draw(self, canvas: list, x: int, y: int):
        """
        Метод, использующий внешнее (extrinsic) состояние (x, y).
        В реальном приложении здесь была бы отрисовка.
        """
        print(f"Drawing a {self._color} {self._name} with texture '{self._texture}' at ({x}, {y})")
        # canvas.draw_tree(x, y, self._texture, self._color) # Примерный вызов

# 2. Flyweight Factory (Фабрика Легковесов)
class TreeFactory:
    """
    Фабрика управляет пулом легковесов (TreeType).
    Она гарантирует, что для одинаковых типов деревьев будет создан только один объект.
    """
    _tree_types: Dict[Tuple[str, str, str], TreeType] = {}

    @classmethod
    def get_tree_type(cls, name: str, color: str, texture: str) -> TreeType:
        """
        Возвращает существующий легковес или создает новый.
        Ключом для пула служит кортеж из intrinsic-свойств.
        """
        key = (name, color, texture)
        if key not in cls._tree_types:
            print(f"FACTORY: Key {key} not found. Creating new TreeType.")
            cls._tree_types[key] = TreeType(name, color, texture)
        else:
            print(f"FACTORY: Key {key} found. Reusing existing TreeType.")
        return cls._tree_types[key]

    @classmethod
    def list_types(cls):
        print(f"\nFACTORY: Current pool has {len(cls._tree_types)} unique tree types:")
        for key in cls._tree_types:
            print(f"  - {key}")

# 3. Client (Клиент - Лес)
class Forest:
    """
    Клиент: использует фабрику для получения легковесов
    и хранит внешнее (extrinsic) состояние для каждого дерева.
    """
    def __init__(self):
        self._trees: list = [] # Хранит {'type': TreeType, 'x': int, 'y': int}

    def plant_tree(self, x: int, y: int, name: str, color: str, texture: str):
        """Сажает дерево, получая общий тип из фабрики."""
        tree_type = TreeFactory.get_tree_type(name, color, texture)
        # Храним только ссылку на легковес и уникальные координаты
        self._trees.append({"type": tree_type, "x": x, "y": y})
        print(f"FOREST: Planted tree at ({x},{y}) using type '{name}'")

    def draw(self):
        """Рисует все деревья, передавая координаты в метод draw легковеса."""
        print("\nFOREST: Drawing all trees...")
        canvas = [] # Имитация холста
        for tree_info in self._trees:
            tree_info["type"].draw(canvas, tree_info["x"], tree_info["y"])

# --- Использование ---
if __name__ == "__main__":
    forest = Forest()

    # Сажаем много деревьев, но типы будут переиспользоваться
    forest.plant_tree(10, 20, "Oak", "Green", "oak_bark.png")
    forest.plant_tree(50, 30, "Birch", "White", "birch_bark.png")
    forest.plant_tree(100, 15, "Oak", "Green", "oak_bark.png") # Переиспользование Oak
    forest.plant_tree(120, 80, "Pine", "Dark Green", "pine_needle.png")
    forest.plant_tree(40, 90, "Oak", "Green", "oak_bark.png") # Снова Oak
    forest.plant_tree(70, 110, "Birch", "White", "birch_bark.png") # Снова Birch
    forest.plant_tree(150, 50, "Oak", "Yellow", "oak_autumn.png") # Новый тип - Желтый дуб

    TreeFactory.list_types() # Показываем, сколько уникальных типов было создано

    forest.draw() # Рисуем лес

# Примерный вывод:
# FACTORY: Key ('Oak', 'Green', 'oak_bark.png') not found. Creating new TreeType.
# FLYWEIGHT: Creating NEW TreeType - Name: Oak, Color: Green, Texture: oak_bark.png
# FOREST: Planted tree at (10,20) using type 'Oak'
# FACTORY: Key ('Birch', 'White', 'birch_bark.png') not found. Creating new TreeType.
# FLYWEIGHT: Creating NEW TreeType - Name: Birch, Color: White, Texture: birch_bark.png
# FOREST: Planted tree at (50,30) using type 'Birch'
# FACTORY: Key ('Oak', 'Green', 'oak_bark.png') found. Reusing existing TreeType.
# FOREST: Planted tree at (100,15) using type 'Oak'
# FACTORY: Key ('Pine', 'Dark Green', 'pine_needle.png') not found. Creating new TreeType.
# FLYWEIGHT: Creating NEW TreeType - Name: Pine, Color: Dark Green, Texture: pine_needle.png
# FOREST: Planted tree at (120,80) using type 'Pine'
# FACTORY: Key ('Oak', 'Green', 'oak_bark.png') found. Reusing existing TreeType.
# FOREST: Planted tree at (40,90) using type 'Oak'
# FACTORY: Key ('Birch', 'White', 'birch_bark.png') found. Reusing existing TreeType.
# FOREST: Planted tree at (70,110) using type 'Birch'
# FACTORY: Key ('Oak', 'Yellow', 'oak_autumn.png') not found. Creating new TreeType.
# FLYWEIGHT: Creating NEW TreeType - Name: Oak, Color: Yellow, Texture: oak_autumn.png
# FOREST: Planted tree at (150,50) using type 'Oak'
#
# FACTORY: Current pool has 4 unique tree types:
#   - ('Oak', 'Green', 'oak_bark.png')
#   - ('Birch', 'White', 'birch_bark.png')
#   - ('Pine', 'Dark Green', 'pine_needle.png')
#   - ('Oak', 'Yellow', 'oak_autumn.png')
#
# FOREST: Drawing all trees...
# Drawing a Green Oak with texture 'oak_bark.png' at (10, 20)
# Drawing a White Birch with texture 'birch_bark.png' at (50, 30)
# Drawing a Green Oak with texture 'oak_bark.png' at (100, 15)
# Drawing a Dark Green Pine with texture 'pine_needle.png' at (120, 80)
# Drawing a Green Oak with texture 'oak_bark.png' at (40, 90)
# Drawing a White Birch with texture 'birch_bark.png' at (70, 110)
# Drawing a Yellow Oak with texture 'oak_autumn.png' at (150, 50)

FACTORY: Key ('Oak', 'Green', 'oak_bark.png') not found. Creating new TreeType.
FLYWEIGHT: Creating NEW TreeType - Name: Oak, Color: Green, Texture: oak_bark.png
FOREST: Planted tree at (10,20) using type 'Oak'
FACTORY: Key ('Birch', 'White', 'birch_bark.png') not found. Creating new TreeType.
FLYWEIGHT: Creating NEW TreeType - Name: Birch, Color: White, Texture: birch_bark.png
FOREST: Planted tree at (50,30) using type 'Birch'
FACTORY: Key ('Oak', 'Green', 'oak_bark.png') found. Reusing existing TreeType.
FOREST: Planted tree at (100,15) using type 'Oak'
FACTORY: Key ('Pine', 'Dark Green', 'pine_needle.png') not found. Creating new TreeType.
FLYWEIGHT: Creating NEW TreeType - Name: Pine, Color: Dark Green, Texture: pine_needle.png
FOREST: Planted tree at (120,80) using type 'Pine'
FACTORY: Key ('Oak', 'Green', 'oak_bark.png') found. Reusing existing TreeType.
FOREST: Planted tree at (40,90) using type 'Oak'
FACTORY: Key ('Birch', 'White', 'birch_bark.png') found. Reusing existing Tree

Сложный пример: Форматирование Символов в Текстовом Редакторе
Представим текстовый редактор, где каждый символ имеет шрифт, размер, начертание (жирный/курсив). Вместо хранения этих атрибутов для каждого символа в документе, мы используем Легковес для представления уникальных комбинаций форматирования.

In [2]:
import weakref # Используем weakref для управления памятью в фабрике (опционально)
from typing import Dict, Tuple, Any

# 1. Flyweight Interface/Class (Стиль Символа)
class CharacterStyle:
    """
    Легковес: хранит неизменяемое (intrinsic) состояние стиля.
    (шрифт, размер, жирность, курсив)
    """
    def __init__(self, font_family: str, font_size: int, is_bold: bool, is_italic: bool):
        print(f"FLYWEIGHT: Creating NEW Style - Font: {font_family}, Size: {font_size}, Bold: {is_bold}, Italic: {is_italic}")
        self.font_family = font_family
        self.font_size = font_size
        self.is_bold = is_bold
        self.is_italic = is_italic
        # В реальном приложении здесь могли бы быть предзагруженные ресурсы шрифтов

    def apply_style(self, character: str, position: int):
        """
        Применяет стиль к символу, используя внешнее (extrinsic) состояние
        (сам символ и его позиция).
        """
        style_str = f"Font='{self.font_family}', Size={self.font_size}"
        if self.is_bold: style_str += ", Bold"
        if self.is_italic: style_str += ", Italic"
        print(f"Rendering char '{character}' at pos {position} with Style ({style_str})")

    # Делаем объект хешируемым для использования в качестве ключа словаря (если нужно)
    # и для сравнения. Важно, чтобы хеш зависел только от intrinsic state.
    def __hash__(self):
        return hash((self.font_family, self.font_size, self.is_bold, self.is_italic))

    def __eq__(self, other):
        if not isinstance(other, CharacterStyle):
            return NotImplemented
        return (self.font_family == other.font_family and
                self.font_size == other.font_size and
                self.is_bold == other.is_bold and
                self.is_italic == other.is_italic)

# 2. Flyweight Factory (Фабрика Стилей)
class StyleFactory:
    """
    Фабрика стилей. Использует weakref.WeakValueDictionary, чтобы стили,
    на которые больше нет ссылок у клиента, могли быть удалены сборщиком мусора.
    """
    _styles: weakref.WeakValueDictionary = weakref.WeakValueDictionary()

    @classmethod
    def get_style(cls, font_family: str, font_size: int, is_bold: bool, is_italic: bool) -> CharacterStyle:
        """Возвращает существующий стиль или создает новый."""
        key = (font_family, font_size, is_bold, is_italic)
        style = cls._styles.get(key)
        if style is None:
            print(f"FACTORY: Style {key} not found or garbage collected. Creating new.")
            style = CharacterStyle(font_family, font_size, is_bold, is_italic)
            cls._styles[key] = style
        else:
            print(f"FACTORY: Style {key} found. Reusing existing.")
        return style

    @classmethod
    def get_pool_size(cls):
        # Реальный размер может быть меньше из-за weakref, если стили уже собраны GC
        return len(cls._styles)

# 3. Client (Клиент - Текстовый Документ)
class TextDocument:
    """
    Клиент: хранит последовательность символов и ссылки на их стили (легковесы).
    """
    def __init__(self):
        # Структура: [(char, style_flyweight_ref), ...]
        self._characters: list[Tuple[str, CharacterStyle]] = []

    def add_character(self, char: str, font_family: str, font_size: int,
                        is_bold: bool = False, is_italic: bool = False):
        """Добавляет символ с указанным стилем в документ."""
        if len(char) != 1:
            raise ValueError("Only single characters allowed")
        style = StyleFactory.get_style(font_family, font_size, is_bold, is_italic)
        self._characters.append((char, style))
        print(f"DOC: Added '{char}' with style {style.font_family}/{style.font_size}{'/B' if style.is_bold else ''}{'/I' if style.is_italic else ''}")

    def render(self):
        """Отображает документ, применяя стили к символам."""
        print("\n--- Rendering Document ---")
        for i, (char, style) in enumerate(self._characters):
            style.apply_style(char, i) # Передаем extrinsic state (char, i)
        print("--- End of Rendering ---")

# --- Использование ---
if __name__ == "__main__":
    doc = TextDocument()

    # Добавляем текст с разным форматированием
    doc.add_character('H', "Arial", 12, is_bold=True)
    doc.add_character('e', "Arial", 12)
    doc.add_character('l', "Arial", 12)
    doc.add_character('l', "Arial", 12) # Стиль Arial 12 будет переиспользован
    doc.add_character('o', "Arial", 12)
    doc.add_character(' ', "Arial", 12)
    doc.add_character('W', "Times New Roman", 14, is_italic=True) # Новый стиль
    doc.add_character('o', "Times New Roman", 14, is_italic=True) # Переиспользование TNR 14 Italic
    doc.add_character('r', "Times New Roman", 14, is_italic=True)
    doc.add_character('l', "Times New Roman", 14, is_italic=True)
    doc.add_character('d', "Times New Roman", 14, is_italic=True)
    doc.add_character('!', "Arial", 12, is_bold=True) # Переиспользование Arial 12 Bold

    print(f"\nTotal characters in document: {len(doc._characters)}")
    print(f"Number of unique styles created (approx. due to weakref): {StyleFactory.get_pool_size()}")

    doc.render()

# Примерный вывод:
# FACTORY: Style ('Arial', 12, True, False) not found or garbage collected. Creating new.
# FLYWEIGHT: Creating NEW Style - Font: Arial, Size: 12, Bold: True, Italic: False
# DOC: Added 'H' with style Arial/12/B
# FACTORY: Style ('Arial', 12, False, False) not found or garbage collected. Creating new.
# FLYWEIGHT: Creating NEW Style - Font: Arial, Size: 12, Bold: False, Italic: False
# DOC: Added 'e' with style Arial/12
# FACTORY: Style ('Arial', 12, False, False) found. Reusing existing.
# DOC: Added 'l' with style Arial/12
# FACTORY: Style ('Arial', 12, False, False) found. Reusing existing.
# DOC: Added 'l' with style Arial/12
# FACTORY: Style ('Arial', 12, False, False) found. Reusing existing.
# DOC: Added 'o' with style Arial/12
# FACTORY: Style ('Arial', 12, False, False) found. Reusing existing.
# DOC: Added ' ' with style Arial/12
# FACTORY: Style ('Times New Roman', 14, False, True) not found or garbage collected. Creating new.
# FLYWEIGHT: Creating NEW Style - Font: Times New Roman, Size: 14, Bold: False, Italic: True
# DOC: Added 'W' with style Times New Roman/14/I
# FACTORY: Style ('Times New Roman', 14, False, True) found. Reusing existing.
# DOC: Added 'o' with style Times New Roman/14/I
# FACTORY: Style ('Times New Roman', 14, False, True) found. Reusing existing.
# DOC: Added 'r' with style Times New Roman/14/I
# FACTORY: Style ('Times New Roman', 14, False, True) found. Reusing existing.
# DOC: Added 'l' with style Times New Roman/14/I
# FACTORY: Style ('Times New Roman', 14, False, True) found. Reusing existing.
# DOC: Added 'd' with style Times New Roman/14/I
# FACTORY: Style ('Arial', 12, True, False) found. Reusing existing.
# DOC: Added '!' with style Arial/12/B
#
# Total characters in document: 12
# Number of unique styles created (approx. due to weakref): 3
#
# --- Rendering Document ---
# Rendering char 'H' at pos 0 with Style (Font='Arial', Size=12, Bold)
# Rendering char 'e' at pos 1 with Style (Font='Arial', Size=12)
# Rendering char 'l' at pos 2 with Style (Font='Arial', Size=12)
# Rendering char 'l' at pos 3 with Style (Font='Arial', Size=12)
# Rendering char 'o' at pos 4 with Style (Font='Arial', Size=12)
# Rendering char ' ' at pos 5 with Style (Font='Arial', Size=12)
# Rendering char 'W' at pos 6 with Style (Font='Times New Roman', Size=14, Italic)
# Rendering char 'o' at pos 7 with Style (Font='Times New Roman', Size=14, Italic)
# Rendering char 'r' at pos 8 with Style (Font='Times New Roman', Size=14, Italic)
# Rendering char 'l' at pos 9 with Style (Font='Times New Roman', Size=14, Italic)
# Rendering char 'd' at pos 10 with Style (Font='Times New Roman', Size=14, Italic)
# Rendering char '!' at pos 11 with Style (Font='Arial', Size=12, Bold)
# --- End of Rendering ---

FACTORY: Style ('Arial', 12, True, False) not found or garbage collected. Creating new.
FLYWEIGHT: Creating NEW Style - Font: Arial, Size: 12, Bold: True, Italic: False
DOC: Added 'H' with style Arial/12/B
FACTORY: Style ('Arial', 12, False, False) not found or garbage collected. Creating new.
FLYWEIGHT: Creating NEW Style - Font: Arial, Size: 12, Bold: False, Italic: False
DOC: Added 'e' with style Arial/12
FACTORY: Style ('Arial', 12, False, False) found. Reusing existing.
DOC: Added 'l' with style Arial/12
FACTORY: Style ('Arial', 12, False, False) found. Reusing existing.
DOC: Added 'l' with style Arial/12
FACTORY: Style ('Arial', 12, False, False) found. Reusing existing.
DOC: Added 'o' with style Arial/12
FACTORY: Style ('Arial', 12, False, False) found. Reusing existing.
DOC: Added ' ' with style Arial/12
FACTORY: Style ('Times New Roman', 14, False, True) not found or garbage collected. Creating new.
FLYWEIGHT: Creating NEW Style - Font: Times New Roman, Size: 14, Bold: False, 